In [1]:
## IMPORTS
%%capture
!pip install h2o
import h2o
from h2o.automl import H2OAutoML
from __future__ import print_function
import pandas_profiling
import pandas as pd
import numpy as np
import os
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

drive.mount('/content/gdrive', )
path_to_project = '/content/gdrive/My Drive/Salary Prediction POC/datadavis-nba-salaries' # Enter the path to your folder here
os.chdir(path_to_project)


# Suppress unwanted warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("requests").setLevel(logging.WARNING)

In [7]:
#### GPU INFO FOR USER (Usually a P100)

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Feb 17 23:25:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Loading data

In [8]:
player_salaries_df = pd.read_csv('player_salaries.csv')
player_salaries_df.drop(columns = ['Unnamed: 0'], inplace = True)
train_ps, test_ps, _, _ = train_test_split(player_salaries_df, player_salaries_df.salary , test_size=0.2, random_state=100)

In [4]:
#### INITIATING THE H2O.AI INSTANCE
%%capture
h2o.init(nthreads=1, max_mem_size="768m")

In [5]:
train = h2o.H2OFrame(python_obj=train_ps)
test = h2o.H2OFrame(python_obj=test_ps)
x= train_ps.columns
y= "salary"

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
train.describe()

Rows:11330
Cols:29




,player_id,salary,season,season_end,season_start,team,birthDate,birthPlace,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,career_PTS,career_TRB,career_WS,career_eFG%,college,draft_pick,draft_round,draft_team,draft_year,height,highSchool,position,shoots,weight,Age
type,enum,int,enum,int,int,enum,enum,enum,real,real,real,real,real,real,real,real,real,real,enum,enum,enum,enum,int,real,enum,enum,enum,real,real
mins,,2706.0,,1985.0,1984.0,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,,1969.0,0.0,,,,0.0,0.0
mean,,3139466.5345101515,,2003.6979699911765,2002.6979699911765,,,,0.17931849703694372,0.45197234982332224,0.2643444993593268,0.7333206859783196,0.3684009363367743,0.4550494245393477,0.30214759275495395,0.2869360460247006,0.1376430249474698,0.3214454063604229,,,,,1996.9703900347135,0.5753877190770396,,,,0.3756515585693114,0.34453459162197114
maxs,,34682550.0,,2018.0,2017.0,,,,1.0,1.0,1.0,1.0,1.0000000000000002,0.912344777209642,1.0,1.0,1.0,1.0,,,,,2017.0,1.0,,,,1.0,1.0000000000000002
sigma,,4168625.034602776,,9.14329416389503,9.14329416389503,,,,0.15323618921330695,0.05929109612300727,0.12997747641683857,0.10122303529475311,0.2196354844641347,0.03105828090969064,0.17113770402924244,0.1673277400528784,0.14118058128105673,0.037256426433312125,,,,,9.897841364103298,0.132234312906261,,,,0.12252443393431031,0.1636790131358264
zeros,,0,,0,0,,,,104,30,1119,11,19,2,35,20,1,30,,,,,0,14,,,,8,10
missing,0,0,0,0,0,0,0,0,0,10,404,76,0,1,0,0,0,10,0,0,0,0,1536,0,0,0,0,0,0
0,roberal01,307500.0,1986-87,1987.0,1986.0,San Antonio Spurs,"July 22, 1962","Barberton, Ohio",0.4464285714285714,0.477,0.295,0.743,0.4832298136645963,0.47918188458729,0.4651162790697674,0.3795620437956205,0.1984788120246287,0.3266666666666666,University of Arkansas,7th overall,1st round,San Antonio Spurs,1984.0,0.4285714285714284,"Barberton in Barberton, Ohio",Shooting Guard,Right,0.2290748898678415,0.2307692307692308
1,poweldw01,845059.0,2015-16,2016.0,2015.0,Dallas Mavericks,"July 20, 1991","Toronto, Canada",0.0803571428571428,0.5529999999999999,0.298,0.75,0.2049689440993788,0.4901387874360847,0.2491694352159468,0.3284671532846716,0.0909090909090909,0.3873333333333333,Stanford University,45th overall,2nd round,Charlotte Hornets,2014.0,0.714285714285714,"IMG Academy in Bradenton, Florida",Power Forward and Center,Right,0.4713656387665198,0.2307692307692308
2,banksma01,4260000.0,2008-09,2009.0,2008.0,Toronto Raptors,"November 19, 1981","Las Vegas, Nevada",0.1875,0.432,0.327,0.768,0.215527950310559,0.4441197954711468,0.1960132890365448,0.1094890510948905,0.0336834480260775,0.3126666666666667,"University of Nevada, Las Vegas",13th overall,1st round,Memphis Grizzlies,2003.0,0.3928571428571428,"Cimarron Memorial in Las Vegas, Nevada",Point Guard,Right,0.2951541850220265,0.3461538461538462


In [9]:
%%capture
# Run AutoML for 1 hours
autoModel = H2OAutoML(max_runtime_secs=(3600 * 1), seed = 100)

autoModel.train(y = y,
            training_frame = train, 
            validation_frame = test, 
            leaderboard_frame = test)

In [15]:
leaders = autoModel.leaderboard
leaders

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_4_AutoML_1_20220217_232551,3.72054e+12,1.92887e+06,3.72054e+12,1.01342e+06,nan
StackedEnsemble_AllModels_4_AutoML_1_20220217_232551,3.75189e+12,1.93698e+06,3.75189e+12,1.00396e+06,nan
StackedEnsemble_AllModels_3_AutoML_1_20220217_232551,3.75419e+12,1.93757e+06,3.75419e+12,1.00339e+06,nan
XGBoost_grid_1_AutoML_1_20220217_232551_model_6,3.91551e+12,1.97876e+06,3.91551e+12,1.0496e+06,nan
GBM_grid_1_AutoML_1_20220217_232551_model_10,3.95032e+12,1.98754e+06,3.95032e+12,1.01167e+06,nan
StackedEnsemble_BestOfFamily_3_AutoML_1_20220217_232551,3.95903e+12,1.98973e+06,3.95903e+12,1.05755e+06,nan
StackedEnsemble_AllModels_2_AutoML_1_20220217_232551,3.9933e+12,1.99832e+06,3.9933e+12,1.06173e+06,nan
StackedEnsemble_BestOfFamily_2_AutoML_1_20220217_232551,4.01488e+12,2.00372e+06,4.01488e+12,1.04008e+06,nan
StackedEnsemble_AllModels_1_AutoML_1_20220217_232551,4.05215e+12,2.01299e+06,4.05215e+12,1.04729e+06,nan
StackedEnsemble_BestOfFamily_1_AutoML_1_20220217_232551,4.13285e+12,2.03294e+06,4.13285e+12,1.07582e+06,nan


In [16]:
### EXAMPLE OF AUTOML BEST MODEL, YOU CAN CHANGE THE CRITERION AS YOU SEE FIT
best_model_automl = autoModel.get_best_model(criterion="rmse")
#### YOU CAN USE THE LINE BELOW TO PREDICT, THE AS.DATAFRAME() CALL SIMPLY PUTS THEM INTO A PANDAS DF FOR YOUR CONVENIENCE
y_pred_automl = best_model_automl.predict(test).as_data_frame()
pred_y = y_pred_automl['predict'].to_numpy()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [17]:
from sklearn.metrics import r2_score
orig_labels = test_ps.salary.to_numpy()
r2_score(orig_labels, pred_y)

0.7938791364658997

In [24]:
(orig_labels - pred_y).mean()

119615.701950872

In [22]:
pd.DataFrame(pred_y, columns = ['salary_pred']).to_csv('salary_predictions.csv')